In [ ]:
import pandas as pd
import numpy as np

# -------------------------------------------------------------------
# LOAD RAW DATASET
# -------------------------------------------------------------------
print("Loading raw dataset...")
df = pd.read_csv("raw_dataset.csv")
df.columns = df.columns.str.lower()

print(f"Dataset shape: {df.shape}")
print(f"Number of unique companies: {df['ticker'].nunique()}")
print(f"Year range: {df['year'].min()} to {df['year'].max()}")

# -------------------------------------------------------------------
# RENAME COLUMNS AND CHECK MISSING VALUES
# -------------------------------------------------------------------
df.rename(columns={"life_cycle_stage": "lifecycle_stage"}, inplace=True)

print("\n🔎 Missing values per column:")
missing = df.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)
print(missing)

# -------------------------------------------------------------------
# LIFECYCLE STAGE DISTRIBUTION BEFORE CLEANING
# -------------------------------------------------------------------
print("\n📈 Lifecycle Stage Distribution:")
lifecycle_counts = df['lifecycle_stage'].value_counts(dropna=False)
print(lifecycle_counts)

print("\nAnalysis complete!")

Loading raw dataset...
Dataset shape: (3870, 15)
Number of unique companies: 774
Year range: 2020 to 2024

🔎 Missing values per column:
ev_sales             3074
r_and_d_ratio        2038
revenue_growth       1737
asset_turnover       1678
net_income_margin    1604
capex_ratio          1579
ocf_margin           1557
roa                  1522
revenue              1519
debt_assets          1455
current_ratio        1246
total_assets         1220
lifecycle_stage       310
dtype: int64

📈 Lifecycle Stage Distribution:
lifecycle_stage
Maturity        1038
Data Missing     821
Introduction     606
Growth           554
Shake-out        334
NaN              310
Decline          207
Name: count, dtype: int64

Analysis complete!


In [ ]:
# -------------------------------------------------------------------
# DATA CLEANING
# -------------------------------------------------------------------
df_cleaned = df.dropna()

if 'lifecycle_stage' in df_cleaned.columns:
    initial_count = df_cleaned.shape[0]
    df_cleaned = df_cleaned[df_cleaned['lifecycle_stage'] != 'Data Missing']
    removed = initial_count - df_cleaned.shape[0]
    print(f"❌ Removed {removed} rows with lifecycle stage 'Data Missing'.")

# -------------------------------------------------------------------
# PRINT OUTPUT
# -------------------------------------------------------------------
print(f"\n✅ Number of observations: {df_cleaned.shape[0]}")
print("\n📊 Life cycle stage distribution:")
print(df_cleaned['lifecycle_stage'].value_counts(dropna=False))

❌ Removed 2 rows with lifecycle stage 'Data Missing'.

✅ Number of observations: 455

📊 Life cycle stage distribution:
lifecycle_stage
Maturity        244
Growth           76
Shake-out        60
Introduction     58
Decline          17
Name: count, dtype: int64


In [ ]:
# -------------------------------------------------------------------
# DESCRIPTIVE STATISTICS BEFORE TRIMMING
# -------------------------------------------------------------------
print("\n📊 Generating descriptive statistics...")

numeric_cols = df_cleaned.select_dtypes(include=[np.number]).columns.tolist()

desc_stats = pd.DataFrame(index=numeric_cols, columns=['N', 'Mean', 'St. Dev.', 'Min', 'Max'])

for col in numeric_cols:
    desc_stats.loc[col, 'N'] = df_cleaned[col].count()
    desc_stats.loc[col, 'Mean'] = df_cleaned[col].mean()
    desc_stats.loc[col, 'St. Dev.'] = df_cleaned[col].std()
    desc_stats.loc[col, 'Min'] = df_cleaned[col].min()
    desc_stats.loc[col, 'Max'] = df_cleaned[col].max()

for col in ['Mean', 'St. Dev.', 'Min', 'Max']:
    desc_stats[col] = desc_stats[col].apply(lambda x: '{:.2f}'.format(float(x)))

desc_stats = desc_stats.reset_index().rename(columns={'index': 'Variable'})

# -------------------------------------------------------------------
# PRINT AND SAVE OUTPUT
# -------------------------------------------------------------------
print(desc_stats)
desc_stats.to_csv("raw_descriptives.csv")
print(f"\n✅ Descriptive statistics saved to raw_descriptives.csv")


📊 Generating descriptive statistics...
             Variable    N            Mean        St. Dev.         Min  \
0                year  455         2022.15            1.40     2020.00   
1             revenue  455  12537578705.78  44510037780.71    41893.00   
2        total_assets  455  19702293091.94  59041207788.36  6411918.00   
3       current_ratio  455            2.35            1.92        0.25   
4         debt_assets  455            0.28            0.24        0.00   
5      asset_turnover  455            0.71            0.41        0.00   
6                 roa  455           -0.02            0.25       -1.60   
7      revenue_growth  455            0.25            1.71       -0.91   
8       r_and_d_ratio  455            0.32            2.70        0.00   
9         capex_ratio  455            0.02            0.03        0.00   
10  net_income_margin  455           -0.74           10.94     -217.94   
11         ocf_margin  455           -0.50            9.79     -201.07  

In [ ]:
import pandas as pd
import numpy as np

# -------------------------------------------------------------------
# CONFIGURATION
# -------------------------------------------------------------------
df = df_cleaned
OUTLIER_TRIM_PERCENT = 1

df.columns = df.columns.str.lower()

# -------------------------------------------------------------------
# IDENTIFY AND REMOVE OUTLIERS
# -------------------------------------------------------------------
print(f"\n✂️ Removing rows with extreme values beyond {OUTLIER_TRIM_PERCENT}% tails...")

numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
keep_mask = pd.Series(True, index=df.index)

for col in numeric_cols:
    lower = np.percentile(df[col].dropna(), OUTLIER_TRIM_PERCENT)
    upper = np.percentile(df[col].dropna(), 100 - OUTLIER_TRIM_PERCENT)

    col_mask = (df[col] >= lower) & (df[col] <= upper)
    removed_count = (~col_mask).sum()

    keep_mask &= col_mask
    print(f"  {col}: removing {removed_count} rows outside [{lower:.2f}, {upper:.2f}]")

df_trimmed = df[keep_mask].reset_index(drop=True)

# -------------------------------------------------------------------
# LIFE CYCLE DISTRIBUTION
# -------------------------------------------------------------------
print("\n📊 Life cycle stage distribution:")
life_cycle_dist = df_trimmed['lifecycle_stage'].value_counts(dropna=False)
print(life_cycle_dist)
life_cycle_dist.to_csv("life_cycle_dist.csv")

# -------------------------------------------------------------------
# MERGE LIFECYCLE STAGES
# -------------------------------------------------------------------
print("🔄 Merging lifecycle stages...")

df_trimmed['lifecycle_stage'] = df_trimmed['lifecycle_stage'].replace({
    'Introduction': 'Early',
    'Growth': 'Early',
    'Shake-out': 'Decline',
    'Decline': 'Decline'
})

# -------------------------------------------------------------------
# COLUMN ORDER
# -------------------------------------------------------------------
first_columns = ['ticker', 'year', 'ev_sales', 'lifecycle_stage', 'roa', 'current_ratio', 'debt_assets', 'asset_turnover']
new_column_order = first_columns + [col for col in df.columns if col not in first_columns]
df = df_trimmed[new_column_order]

# -------------------------------------------------------------------
# PRINT AND SAVE CLEANED DATASET AND LIFE CYCLE DISTRIBUTIONS
# -------------------------------------------------------------------
print(f"\n✅ Final number of observations: {df_trimmed.shape[0]}")
print("\n📊 Life cycle stage distribution after merging:")
life_cycle_dist_final = df_trimmed['lifecycle_stage'].value_counts(dropna=False)
print(life_cycle_dist_final)

df.to_csv("cleaned_dataset.csv", index=False)
life_cycle_dist_final.to_csv("life_cycle_dist_final.csv")
print(f"\n✅ Trimmed dataset saved to cleaned_dataset.csv")


✂️ Removing rows with extreme values beyond 1% tails...
  year: removing 0 rows outside [2020.00, 2024.00]
  revenue: removing 10 rows outside [3825740.08, 258642780000.00]
  total_assets: removing 10 rows outside [8811059.04, 352662120000.00]
  current_ratio: removing 10 rows outside [0.46, 9.53]
  debt_assets: removing 10 rows outside [0.00, 1.10]
  asset_turnover: removing 10 rows outside [0.09, 1.93]
  roa: removing 10 rows outside [-1.27, 0.42]
  revenue_growth: removing 10 rows outside [-0.50, 4.14]
  r_and_d_ratio: removing 10 rows outside [0.00, 1.44]
  capex_ratio: removing 10 rows outside [0.00, 0.13]
  net_income_margin: removing 10 rows outside [-4.80, 0.61]
  ocf_margin: removing 10 rows outside [-2.99, 0.50]
  ev_sales: removing 10 rows outside [-0.00, 214.74]

📊 Life cycle stage distribution:
lifecycle_stage
Maturity        212
Growth           67
Shake-out        49
Introduction     39
Decline          12
Name: count, dtype: int64
🔄 Merging lifecycle stages...

✅ Final